In [78]:
#necessary imports:
import geopandas as gpd
import pandas as pd
import random
import sys
sys.path.append('../')
import modules.predict as predict
import modules.encoder as encoder
import copy
import numpy as np

In [79]:
gdf = gpd.read_file('./import/TEST_BEFORE_COMPLETION.geojson')

In [70]:

#before performing the mapping we want to fill incomplete columns 
target_columns = ['numberOfFloors_original', 'rooftype_original']
selected_predictors = ['class', 'first_year', 'last_year', 'numberOfFloors_original', 'material.value', 'rooftype_original']
gdf_new = copy.deepcopy(gdf)
for target in target_columns:
    filled_column, is_pred_column = predict.fill_missing_values(target, gdf, selected_predictors)
    #gdf[target] = filled_column
    #put the filled column in the gdf substituting the original one
    gdf_new[target] = filled_column
    gdf_new[target + '_type'] = is_pred_column

In [71]:
# specify in the mapping which columns should change name. 
mapping = {
    'type': 'class',
    'time.estimatedStart.timeMoment.year': 'first_year',    
    'time.estimatedEnd.timeMoment.year': 'last_year',  
    'numberOfFloors.value':'numberOfFloors_original',
    'numberOfFloors.paradata.type':'numberOfFloors_original_type',
    'roof.type.value': 'rooftype_original',
    'roof.type.paradata.type': 'rooftype_original_type'
}
# on the left, the name of the field according to the Historical CityJSON extension. On the right, the name of the field in the input geodata.
# The fields that are already correctly encoded don't need to be inserted in the mapping
# it's not necessary to eliminate non mappable fields, when creating the cityjson they will be ignored

# Read the CSV file with the fields to check
fields_df = pd.read_csv('../extension/geojson_mapping.csv')
# Map the GeoDataFrame to HistoricalCityJSON
mapped_gdf = predict.map_gdf_to_historicalcityjson(gdf_new, fields_df, mapping)

mapped_gdf['roof.type.value'].unique()


array(['gabled', 'flat', 'cupola', 'destroyed building'], dtype=object)

In [72]:
gdf= mapped_gdf
for row in gdf.iterrows():
    if row[1]['roof.type.value'] == 'gabled' or row[1]['roof.type.value'] == 'slanted':
        row[1]['roof.type.value'] = random.choice(['hip','gable'])
        gdf.loc[row[0],'roof.type.value'] = row[1]['roof.type.value']
    if row[1]['numberOfFloors.value'] == '1-3':
        row[1]['numberOfFloors.value'] = random.choice([1,2,3])
        gdf.loc[row[0],'numberOfFloors.value'] = row[1]['numberOfFloors.value']
    if row[1]['numberOfFloors.value'] == '4+':
        row[1]['numberOfFloors.value'] = random.choice([4,5,6])
        gdf.loc[row[0],'numberOfFloors.value'] = row[1]['numberOfFloors.value']
    if row[1]['numberOfFloors.value'] == '6+':
        row[1]['numberOfFloors.value'] = random.choice([6,7,8])
        gdf.loc[row[0],'numberOfFloors.value'] = row[1]['numberOfFloors.value']
    if row[1]['type'] == 'archway':
        row[1]['type'] = 'Archway'
        gdf.loc[row[0],'type'] = row[1]['type']

mapping_type = {'archway':'Archway'}
mapping_rooftype = {
    'destroyed building': '',
    'cupola':'domed',
                    }



In [74]:
df_mapped_values = encoder.map_column_values(gdf, 'type', mapping_type)
df_mapped_values = encoder.map_column_values(df_mapped_values, 'roof.type.value', mapping_rooftype)

In [75]:
df_mapped_values['roof.type.value'].unique()

array(['gable', 'flat', 'hip', 'domed', ''], dtype=object)

In [77]:
#save the dataframe as a geojson
df_mapped_values.to_file('./import/TEST_AFTER_COMPLETION.geojson', driver='GeoJSON')